In [2]:
from __future__ import absolute_import
from matplotlib import pyplot as plt

import os
import tensorflow as tf
import numpy as np
import random
import math
from preprocess import get_data
from sklearn import preprocessing
from main import *

2023-05-03 16:16:36.088677: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
#Get data and reshape

#Import and reshape
X_train, X_test, X_val, Y_train, Y_test, Y_val = get_data(.3, "./../processed_data/")

char_encoder = preprocessing.LabelEncoder().fit(np.concatenate((Y_train.reshape(-1), Y_test.reshape(-1), Y_val.reshape(-1))))
ohe_size = np.max(char_encoder.transform(np.concatenate((Y_train.reshape(-1), Y_test.reshape(-1), Y_val.reshape(-1)))))

orig_shapes = [Y_train.shape[0], Y_test.shape[0], Y_val.shape[0]]
Y_train = char_encoder.transform(Y_train.reshape(-1)).reshape((orig_shapes[0], 4))
Y_test = char_encoder.transform(Y_test.reshape(-1)).reshape((orig_shapes[1], 4))
Y_val = char_encoder.transform(Y_val.reshape(-1)).reshape((orig_shapes[2], 4))

#Convert input to properly shaped and typed tensors
X_train = tf.convert_to_tensor(np.asarray(np.reshape(X_train, (-1, *X_train.shape[-2:]))).astype(np.float32))
X_val   = tf.convert_to_tensor(np.asarray(np.reshape(X_val  , (-1, *X_val.shape[-2:]))).astype(np.float32))
Y_train = tf.convert_to_tensor(np.asarray(np.reshape(Y_train, (-1))))
Y_val   = tf.convert_to_tensor(np.asarray(np.reshape(Y_val  , (-1))))

#Don't reshape test, because we need to preserve CAPTCHAs but convert to tensor
X_test  = tf.convert_to_tensor(np.asarray(X_test).astype(np.float32))
Y_test  = tf.convert_to_tensor(np.asarray(Y_test))

#Expand Dims
X_train = tf.expand_dims(X_train, axis=-1)
X_val   = tf.expand_dims(X_val  , axis=-1)

#One hot encode
Y_val   = tf.one_hot(Y_val  , depth=ohe_size, axis=-1)
Y_train = tf.one_hot(Y_train, depth=ohe_size, axis=-1)

Z


IndexError: index 32 is out of bounds for axis 0 with size 32

In [13]:
#model = Model(input_shape=(X_train.shape[-3], X_train.shape[-2], X_train.shape[-1]), num_classes=ohe_size)
model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(32, (3,3), padding="same", input_shape=(X_train.shape[-3], X_train.shape[-2], X_train.shape[-1])),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),
            tf.keras.layers.MaxPool2D(pool_size=(2,2)),

            tf.keras.layers.Conv2D(64, (3,3), padding="same", activation="leaky_relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),
            tf.keras.layers.MaxPool2D(pool_size=(2,2)),

            tf.keras.layers.Conv2D(128, (3,3), padding="same", activation="leaky_relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),
            tf.keras.layers.MaxPool2D(pool_size=(2,2)),

            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(256, activation='leaky_relu'),
            tf.keras.layers.Dense(ohe_size)
        ])
model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(.0001))

In [ ]:
#Fit and save model

model.fit(
    X_train,
    Y_train,
    epochs=4,
    batch_size=256,
    validation_data=(X_val, Y_val)
)

In [9]:
model.save('./../models/segmented_2', save_format="h5")

In [11]:
model = tf.keras.models.load_model("./../models/segmented_2")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 24, 24, 32)       128       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 12, 12, 64)       2

In [12]:
#Load a model and run tests to get accuracy and print an example

#TODO: I don't know how to properly load and save models, it's not working

def get_acc(X_test, Y_test):
    X_test    = np.reshape(X_test, (-1, *X_test.shape[-2:]))
    Y_test    = np.reshape(Y_test, (-1))
    
    probs     = model.predict(X_test, verbose=0)
    diff      = np.argmax(probs, axis=1) - Y_test
    char_acc  = np.where(diff == 0, 1, 0)

    captcha_acc = np.array_split(diff, diff.shape[0]//4)
    captcha_acc = np.apply_along_axis((lambda x: 1 if np.all(x == 0) else 0), axis=1, arr=captcha_acc)

    return np.mean(char_acc), np.mean(captcha_acc)
    

output = model.predict(X_test[7], verbose=0)

print("Secret CAPTCHA:")
print(char_encoder.inverse_transform(Y_test[7]))

print("Model guess:")
print(char_encoder.inverse_transform(np.argmax(output, axis=1)))

char_acc, captcha_acc = get_acc(X_test, Y_test)
print(f"Character accuracy: {round(100*char_acc, 2)}% \nCaptcha accuracy: {round(100*captcha_acc, 2)}%")


Secret CAPTCHA:
['2' '5' 'M' 'Z']
Model guess:
['3' 'B' 'B' 'B']
Character accuracy: 3.45% 
Captcha accuracy: 0.0%
